<a href="https://colab.research.google.com/github/0xpix/CERI-Project/blob/main/notebooks/Geemap_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Earth Engine

In [3]:
# @title Important packages
%%capture
!pip install -U geemap # Upgrade geemap to the latest version

#import Libraries
import ee
import geemap

# GEE authentication
ee.Authenticate()
ee.Initialize(project='aims-ai2324-std-aness-9fl')

In [4]:
geemap.__version__ # geemap version

'0.32.0'

In [5]:
''' Customizing the map
center ([lat, lon]), zoom, width, and height.
data_ctrl=False, toolbar_ctrl=False, draw_ctrl=False
'''
m = geemap.Map(center=[40, -100], zoom=3.5, height=900, width='50%', toolbar_ctrl=False, draw_ctrl=False) # Create a map
m # Display the map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

## Adding basemaps

There are several ways to add basemaps to a map. You can specify the basemap to use in the basemap keyword argument when creating the map. Alternatively, you can add basemap layers to the map using the add_basemap method. Geemap has hundreds of built-in basemaps available that can be easily added to the map with only one line of code.

Create a map by specifying the basemap to use as follows. For example, the Esri.WorldImagery basemap represents the Esri world imagery basemap.

In [6]:
m = geemap.Map(basemap="Esri.WorldImagery", width='50%')
m.add_basemap("Esri.WorldTopoMap")
m.add_basemap("OpenTopoMap")

In [7]:
basemaps = list(geemap.basemaps.keys())
len(geemap.basemaps)

145

In [8]:
basemaps[:10]

['OpenStreetMap',
 'Esri.WorldStreetMap',
 'Esri.WorldImagery',
 'Esri.WorldTopoMap',
 'FWS NWI Wetlands',
 'FWS NWI Wetlands Raster',
 'NLCD 2021 CONUS Land Cover',
 'NLCD 2019 CONUS Land Cover',
 'NLCD 2016 CONUS Land Cover',
 'NLCD 2013 CONUS Land Cover']